In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests


In [24]:
dataset = pd.read_csv('Webpages_Classification_train_data.csv')
dataset.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good
1,1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good
2,2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
3,3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad
4,4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good


In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 12 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1200000 non-null  int64  
 1   url         1200000 non-null  object 
 2   url_len     1200000 non-null  int64  
 3   ip_add      1200000 non-null  object 
 4   geo_loc     1200000 non-null  object 
 5   tld         1200000 non-null  object 
 6   who_is      1200000 non-null  object 
 7   https       1200000 non-null  object 
 8   js_len      1200000 non-null  float64
 9   js_obf_len  1200000 non-null  float64
 10  content     1200000 non-null  object 
 11  label       1200000 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 109.9+ MB


In [26]:
dataset["label"].value_counts()

label
good    1172747
bad       27253
Name: count, dtype: int64

In [27]:
# check for repeted website, no need to check multiple pages just the main link is needed
dataset["url"].value_counts()

url
http://www.hiddenforest.co.nz/fungi/family/pluteaceae/amanita.htm    6
http://www.fcc.gov/                                                  5
http://cyber.law.harvard.edu/                                        5
http://www.symbols.com/                                              5
http://omacl.org/                                                    5
                                                                    ..
http://www.falconups.com/                                            1
http://www.angelfire.com/ga2/ewanlinks/main.html                     1
http://www.unidata.ucar.edu/packages/dods/                           1
http://www.geocities.com/sunsetstrip/birdland/8001/football.html     1
http://homepages.gotadsl.co.uk/~jgm/ekmm/                            1
Name: count, Length: 1171470, dtype: int64

In [33]:
# trim the url to get the main link (http or https, domain) any thing after the / is should be trimmed use regex r'^(https?://)?([\da-z\.-]+)\.([a-z\.]{2,6})'
import re
def extract_base_url(url):
    # This regex will match the protocol (http/https), domain, and top-level domain
    regex = r'^(https?://)?([\da-z\.-]+)\.([a-z\.]{2,6})'
    match = re.match(regex, url)
    if match:
        # This will concatenate the matched groups to form the base URL
        return match.group(0)
    else:
        return None

dataset["base_url"] = dataset["url"].apply(extract_base_url)
dataset.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label,base_url
0,0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,http://members.tripod.com
1,1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,http://www.ddj.com
2,2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,http://www.naef-usa.com
3,3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,http://www.ff-b2b.de
4,4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,http://us.imdb.com


In [35]:
# get the base_url and label only
dataset = dataset[["base_url", "label"]]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   base_url  1198752 non-null  object
 1   label     1200000 non-null  object
dtypes: object(2)
memory usage: 18.3+ MB


In [36]:
# remove duplicates
dataset = dataset.drop_duplicates()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 760595 entries, 0 to 1199999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   base_url  760593 non-null  object
 1   label     760595 non-null  object
dtypes: object(2)
memory usage: 17.4+ MB


In [38]:
dataset["label"].value_counts()

label
good    742332
bad      18263
Name: count, dtype: int64

In [39]:
# balance the dataset since label good is more than label bad
dataset_good = dataset[dataset["label"] == "good"]
dataset_bad = dataset[dataset["label"] == "bad"]
dataset_good = dataset_good.sample(dataset_bad.shape[0])
dataset = pd.concat([dataset_good, dataset_bad])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36526 entries, 585006 to 1199910
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   base_url  36525 non-null  object
 1   label     36526 non-null  object
dtypes: object(2)
memory usage: 856.1+ KB


In [40]:
# check for null values
dataset.isnull().sum()

# remove null values
dataset = dataset.dropna()

base_url    1
label       0
dtype: int64

In [41]:
# the final dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36526 entries, 585006 to 1199910
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   base_url  36525 non-null  object
 1   label     36526 non-null  object
dtypes: object(2)
memory usage: 856.1+ KB
